In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy
# Import the requests library.
import requests
# Import the API key.
from config import weather_api_key
import sys
from datetime import datetime
import traceback

In [2]:
# Create a set of random latitude and longitude combinations.
num_lats = 1500
lats = np.random.uniform(low=-90.000, high=90.000, size=num_lats)
lngs = np.random.uniform(low=-180.000, high=180.000, size=num_lats)
lat_lngs = zip(lats, lngs)
len(lats) 

1500

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
#Sample return from this city url
'''
{'coord': {'lon': 20.04, 'lat': -34.53}, 
'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 
'base': 'stations', 
'main': {'temp': 68.49, 'feels_like': 65.84, 'temp_min': 68, 'temp_max': 69.01, 'pressure': 1014, 'humidity': 60}, 
'wind': {'speed': 6.93, 'deg': 160}, 
'clouds': {'all': 0}, 
'dt': 1587227625, 
'sys': {'type': 1, 'id': 1966, 'country': 'ZA', 'sunrise': 1587186325, 'sunset': 1587226343}, 
'timezone': 7200, 'id': 1015776, 'name': 'Bredasdorp', 'cod': 200}
'''
#Helper method used to create the URL to make an API to openweathermap
def create_city_url(c_name):
    # Starting URL for Weather Map API Call.
    units = "Imperial"
    api_url = "http://api.openweathermap.org/data/2.5/weather"
    city_url = f"{api_url}?units={units}&APPID={weather_api_key}&q={c_name}"
    return city_url

In [5]:
weather_list=[]
#Loop through each lat/long pair
for coordinate in coordinates:
    #Utilize citipy to get the nearest cities
    nearest_city = citipy.nearest_city(coordinate[0], coordinate[1])
    c_name = nearest_city.city_name
    c_country_code = nearest_city.country_code
    city_url = create_city_url(c_name)
    try:
        print(f"Processing city: {c_name}")
        #Make API request to get data on city
        city_weather = requests.get(city_url)
        #HTTP 200 is success
        if city_weather.status_code == 200:
            #Convert response to JSON
            city_weather_json = city_weather.json()
            #Parse out each relevant data element from the JSON structure
            lat = city_weather_json["coord"]["lat"]
            lng = city_weather_json["coord"]["lon"]
            max_temp = city_weather_json["main"]["temp_max"]
            humidity = city_weather_json["main"]["humidity"]
            cloudiness = city_weather_json["clouds"]["all"]
            wind_speed = city_weather_json["wind"]["speed"]
            description = city_weather_json["weather"][0]["description"]
            #Get the timestamp and convert it into a datetime object
            dt_object = datetime.fromtimestamp(city_weather_json["dt"])
            date_time = dt_object.strftime("%m/%d/%Y, %H:%M:%S")
            #Attempt to access the rain element
            try:
                city_rain = city_weather_json["rain"]["3h"]
            except (KeyError,NameError) as e:
                #Rain key does not exist so set the rain to 0
                city_rain = 0
                print(f"Error: Rain processing city {c_name}: Exception: {str(e)}")
                pass
            #Attempt to access the snow element
            try:
                city_snow = city_weather_json["snow"]["3h"]
                print(f"Found snow: {city_snow}")
            except (KeyError,NameError) as e:
                #Snow key does not exist so set the rain to 0
                city_snow = 0
                print(f"Key Error: Snow processing city: {c_name}: Exception: {str(e)}")
                pass
            weather_dict = {
                "City": c_name,
                "Country": c_country_code,
                "Date": date_time,
                "Lat": lat,
                "Lng": lng,
                "Max Temp": max_temp,
                "Humidity": humidity,
                "Cloudiness": cloudiness,
                "Wind Speed": wind_speed,
                "Current Description": description,
                "Rain inches (last 3 hrs)": city_rain,
                "Snow inches (last 3 hrs)": city_snow
            }
            weather_list.append(weather_dict)
        else:
            print(f"City Weather not found city: {c_name}")
    except AttributeError as ae:
        print(f"Error processing city: {c_name} AttributeError:{str(ae)}")
    except Exception as e:
        print(f"Error processing city {c_name} API Request:{str(e)}")

Processing city: severo-kurilsk
Error: Rain processing city severo-kurilsk: Exception: 'rain'
Key Error: Snow processing city: severo-kurilsk: Exception: 'snow'
Processing city: bluff
Key Error: Snow processing city: bluff: Exception: 'snow'
Processing city: barentsburg
City Weather not found city: barentsburg
Processing city: samarai
Key Error: Snow processing city: samarai: Exception: 'snow'
Processing city: rikitea
Error: Rain processing city rikitea: Exception: 'rain'
Key Error: Snow processing city: rikitea: Exception: 'snow'
Processing city: faanui
Key Error: Snow processing city: faanui: Exception: 'snow'
Processing city: mount gambier
Error: Rain processing city mount gambier: Exception: 'rain'
Key Error: Snow processing city: mount gambier: Exception: 'snow'
Processing city: albany
Key Error: Snow processing city: albany: Exception: 'snow'
Processing city: sentyabrskiy
City Weather not found city: sentyabrskiy
Processing city: santa maria
Error: Rain processing city santa mari

Error: Rain processing city codrington: Exception: 'rain'
Key Error: Snow processing city: codrington: Exception: 'snow'
Processing city: qafsah
City Weather not found city: qafsah
Processing city: grand gaube
Error: Rain processing city grand gaube: Exception: '3h'
Key Error: Snow processing city: grand gaube: Exception: 'snow'
Processing city: kyzyl-suu
Error: Rain processing city kyzyl-suu: Exception: 'rain'
Key Error: Snow processing city: kyzyl-suu: Exception: 'snow'
Processing city: mehamn
Error: Rain processing city mehamn: Exception: 'rain'
Key Error: Snow processing city: mehamn: Exception: 'snow'
Processing city: kristinehamn
Error: Rain processing city kristinehamn: Exception: 'rain'
Key Error: Snow processing city: kristinehamn: Exception: 'snow'
Processing city: ende
Error: Rain processing city ende: Exception: 'rain'
Key Error: Snow processing city: ende: Exception: 'snow'
Processing city: lienz
Key Error: Snow processing city: lienz: Exception: 'snow'
Processing city: av

Error: Rain processing city omsukchan: Exception: 'rain'
Found snow: 0.41
Processing city: ziarat
Error: Rain processing city ziarat: Exception: 'rain'
Key Error: Snow processing city: ziarat: Exception: 'snow'
Processing city: taolanaro
City Weather not found city: taolanaro
Processing city: dikson
Error: Rain processing city dikson: Exception: 'rain'
Key Error: Snow processing city: dikson: Exception: 'snow'
Processing city: havoysund
Error: Rain processing city havoysund: Exception: 'rain'
Key Error: Snow processing city: havoysund: Exception: 'snow'
Processing city: hermanus
Error: Rain processing city hermanus: Exception: 'rain'
Key Error: Snow processing city: hermanus: Exception: 'snow'
Processing city: praia
Error: Rain processing city praia: Exception: 'rain'
Key Error: Snow processing city: praia: Exception: 'snow'
Processing city: albany
Key Error: Snow processing city: albany: Exception: 'snow'
Processing city: butaritari
Error: Rain processing city butaritari: Exception: '

Processing city: balkhash
Error: Rain processing city balkhash: Exception: 'rain'
Key Error: Snow processing city: balkhash: Exception: 'snow'
Processing city: chemax
Error: Rain processing city chemax: Exception: 'rain'
Key Error: Snow processing city: chemax: Exception: 'snow'
Processing city: santa eulalia del rio
City Weather not found city: santa eulalia del rio
Processing city: punta arenas
Error: Rain processing city punta arenas: Exception: 'rain'
Key Error: Snow processing city: punta arenas: Exception: 'snow'
Processing city: centralia
Error: Rain processing city centralia: Exception: 'rain'
Key Error: Snow processing city: centralia: Exception: 'snow'
Processing city: phan thiet
Error: Rain processing city phan thiet: Exception: 'rain'
Key Error: Snow processing city: phan thiet: Exception: 'snow'
Processing city: erzin
Error: Rain processing city erzin: Exception: 'rain'
Key Error: Snow processing city: erzin: Exception: 'snow'
Processing city: punta arenas
Error: Rain proc

Error: Rain processing city ridgecrest: Exception: 'rain'
Key Error: Snow processing city: ridgecrest: Exception: 'snow'
Processing city: alcudia
Key Error: Snow processing city: alcudia: Exception: 'snow'
Processing city: jian
Key Error: Snow processing city: jian: Exception: 'snow'
Processing city: la asuncion
Error: Rain processing city la asuncion: Exception: 'rain'
Key Error: Snow processing city: la asuncion: Exception: 'snow'
Processing city: kidal
Error: Rain processing city kidal: Exception: 'rain'
Key Error: Snow processing city: kidal: Exception: 'snow'
Processing city: itarema
Error: Rain processing city itarema: Exception: 'rain'
Key Error: Snow processing city: itarema: Exception: 'snow'
Processing city: dalbandin
Error: Rain processing city dalbandin: Exception: 'rain'
Key Error: Snow processing city: dalbandin: Exception: 'snow'
Processing city: kodiak
Error: Rain processing city kodiak: Exception: 'rain'
Key Error: Snow processing city: kodiak: Exception: 'snow'
Proces

...

City Weather not found city: illoqqortoormiut
Processing city: hithadhoo
Error: Rain processing city hithadhoo: Exception: 'rain'
Key Error: Snow processing city: hithadhoo: Exception: 'snow'
Processing city: saint george
Error: Rain processing city saint george: Exception: 'rain'
Key Error: Snow processing city: saint george: Exception: 'snow'
Processing city: pangai
Error: Rain processing city pangai: Exception: 'rain'
Key Error: Snow processing city: pangai: Exception: 'snow'
Processing city: torbay
Error: Rain processing city torbay: Exception: '3h'
Key Error: Snow processing city: torbay: Exception: 'snow'
Processing city: sentyabrskiy
City Weather not found city: sentyabrskiy
Processing city: saint-philippe
Error: Rain processing city saint-philippe: Exception: 'rain'
Key Error: Snow processing city: saint-philippe: Exception: 'snow'
Processing city: jamestown
Error: Rain processing city jamestown: Exception: 'rain'
Key Error: Snow processing city: jamestown: Exception: 'snow'
Pr

Error: Rain processing city nara: Exception: 'rain'
Key Error: Snow processing city: nara: Exception: 'snow'
Processing city: great falls
Error: Rain processing city great falls: Exception: 'rain'
Key Error: Snow processing city: great falls: Exception: 'snow'
Processing city: ushuaia
Error: Rain processing city ushuaia: Exception: 'rain'
Key Error: Snow processing city: ushuaia: Exception: 'snow'
Processing city: hithadhoo
Error: Rain processing city hithadhoo: Exception: 'rain'
Key Error: Snow processing city: hithadhoo: Exception: 'snow'
Processing city: port elizabeth
Error: Rain processing city port elizabeth: Exception: 'rain'
Key Error: Snow processing city: port elizabeth: Exception: 'snow'
Processing city: hilo
Error: Rain processing city hilo: Exception: 'rain'
Key Error: Snow processing city: hilo: Exception: 'snow'
Processing city: kodiak
Error: Rain processing city kodiak: Exception: 'rain'
Key Error: Snow processing city: kodiak: Exception: 'snow'
Processing city: tcholli

Error: Rain processing city pisco: Exception: 'rain'
Key Error: Snow processing city: pisco: Exception: 'snow'
Processing city: sergeyevka
Key Error: Snow processing city: sergeyevka: Exception: 'snow'
Processing city: bethel
Error: Rain processing city bethel: Exception: 'rain'
Key Error: Snow processing city: bethel: Exception: 'snow'
Processing city: solnechnyy
Error: Rain processing city solnechnyy: Exception: 'rain'
Key Error: Snow processing city: solnechnyy: Exception: 'snow'
Processing city: port blair
Key Error: Snow processing city: port blair: Exception: 'snow'
Processing city: khatanga
Error: Rain processing city khatanga: Exception: 'rain'
Key Error: Snow processing city: khatanga: Exception: 'snow'
Processing city: cidreira
Error: Rain processing city cidreira: Exception: 'rain'
Key Error: Snow processing city: cidreira: Exception: 'snow'
Processing city: dingle
Key Error: Snow processing city: dingle: Exception: 'snow'
Processing city: ushuaia
Error: Rain processing city

Error: Rain processing city georgetown: Exception: 'rain'
Key Error: Snow processing city: georgetown: Exception: 'snow'
Processing city: canoinhas
Error: Rain processing city canoinhas: Exception: 'rain'
Key Error: Snow processing city: canoinhas: Exception: 'snow'
Processing city: provideniya
Error: Rain processing city provideniya: Exception: 'rain'
Found snow: 0.32
Processing city: kaitangata
Error: Rain processing city kaitangata: Exception: 'rain'
Key Error: Snow processing city: kaitangata: Exception: 'snow'
Processing city: hilo
Error: Rain processing city hilo: Exception: 'rain'
Key Error: Snow processing city: hilo: Exception: 'snow'
Processing city: illoqqortoormiut
City Weather not found city: illoqqortoormiut
Processing city: albany
Key Error: Snow processing city: albany: Exception: 'snow'
Processing city: tsihombe
City Weather not found city: tsihombe
Processing city: alofi
Key Error: Snow processing city: alofi: Exception: 'snow'
Processing city: oyama
Key Error: Snow p

Error: Rain processing city mayo: Exception: 'rain'
Key Error: Snow processing city: mayo: Exception: 'snow'
Processing city: shimoda
Error: Rain processing city shimoda: Exception: 'rain'
Key Error: Snow processing city: shimoda: Exception: 'snow'
Processing city: cidreira
Error: Rain processing city cidreira: Exception: 'rain'
Key Error: Snow processing city: cidreira: Exception: 'snow'
Processing city: new norfolk
Key Error: Snow processing city: new norfolk: Exception: 'snow'
Processing city: rikitea
Error: Rain processing city rikitea: Exception: 'rain'
Key Error: Snow processing city: rikitea: Exception: 'snow'
Processing city: hilo
Error: Rain processing city hilo: Exception: 'rain'
Key Error: Snow processing city: hilo: Exception: 'snow'
Processing city: busselton
Key Error: Snow processing city: busselton: Exception: 'snow'
Processing city: torbay
Error: Rain processing city torbay: Exception: '3h'
Key Error: Snow processing city: torbay: Exception: 'snow'


In [6]:
weather_df = pd.DataFrame(weather_list)
weather_df.head(20)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,severo-kurilsk,ru,"04/19/2020, 23:06:14",50.68,156.12,32.20,89,46,9.78,scattered clouds,0.00,0.0
1,bluff,nz,"04/19/2020, 23:06:14",-46.60,168.33,54.00,79,63,5.01,light rain,0.38,0.0
2,samarai,pg,"04/19/2020, 23:06:14",-10.62,150.67,84.69,71,67,8.46,light rain,0.16,0.0
3,rikitea,pf,"04/19/2020, 23:06:15",-23.12,-134.97,76.80,71,61,6.62,broken clouds,0.00,0.0
4,faanui,pf,"04/19/2020, 23:02:14",-16.48,-151.75,82.96,76,89,21.74,light rain,0.24,0.0
5,mount gambier,au,"04/19/2020, 23:06:15",-37.83,140.77,60.80,63,75,14.99,broken clouds,0.00,0.0
6,albany,au,"04/19/2020, 23:06:16",42.60,-73.97,48.00,57,96,8.14,light rain,0.30,0.0
7,santa maria,cv,"04/19/2020, 23:02:12",-29.68,-53.81,59.02,76,0,3.44,clear sky,0.00,0.0
8,hermanus,za,"04/19/2020, 23:06:16",-34.42,19.23,61.00,93,11,1.01,few clouds,0.00,0.0
9,baraya,co,"04/19/2020, 23:06:17",3.15,-75.05,70.07,57,18,4.18,light rain,1.03,0.0


In [7]:
#Save the dataframe to file.
weather_df.to_csv (r'./analysis/WeatherPy_challenge.csv', index = False, header=True)

In [8]:
#The number of coordinate pairs that were found from openweathermap API (originally 1,500)
len(weather_df)

1371

In [9]:
#The number of coordinate pairs (from openweathermap API) to have rained in the last 3hours
total_rain = len(weather_df[weather_df["Rain inches (last 3 hrs)"]>0])

In [10]:
#The number of coordinate pairs (from openweathermap API) to have snowed in the last 3hours
total_snow=len(weather_df[weather_df["Snow inches (last 3 hrs)"]>0])

In [11]:
#Total number of cities with either rain or snow in the last 3 hours.
total_rain_snow = total_rain + total_snow
total_rain_snow

247